In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy
import pandas as pd

import sys
sys.path.append("../../")
import os
from src.functional import free_gpu_cache
from src.utils import env_utils

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src.models import load_LM
import torch

# model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
model_key = "meta-llama/Meta-Llama-3-70B-Instruct"

lm = load_LM(
    model_key=model_key,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 30/30 [01:02<00:00,  2.08s/it]

loaded /home/local_arnab/Codes/00_MODEL/meta-llama/Meta-Llama-3-70B-Instruct | size: 36650.535 MB


In [4]:
from src.dataset import SampleV3, DatasetV3
from src.dataset import STORY_TEMPLATES
from src.functional import predict_next_token

STORY_TEMPLATES["templates"][0]

{'context': '<character_1> is working in a busy restaurant. A customer asks <character_1> for <state_1>. <character_1> grabs an opaque <container_1> and fills it with <state_1>. Then <character_1> grabs another opaque <container_2> and fills it with <state_2>.',
 'causal_event': "A co-worker named <character_2> didn't hear the customer's request and swaps the <state_event> in the <container_event> with <state_swap> while <character_1> was attending to another task.",
 'event_noticed': "<character_1> <noticed/didn't notice> <character_2> swapping the contents of the <container_event>.",
 'question': 'Does <character_q> believe <container_q> contains <state_q>?',
 'container_type': 'bottles',
 'state_type': 'drinks'}

In [5]:
STORY_TEMPLATES["placeholders"]["event"]["container_event"]

'<container_event>'

In [6]:
sample = SampleV3(
    template = STORY_TEMPLATES["templates"][0],
    characters=["Alice", "Bob"],
    containers=["mug", "cup"],
    states=["milk", "juice"],
    event_idx = None, # causal event didn't happen
)

print(sample.character_belief)

d = DatasetV3(samples=[sample])
prompt, answer = d.__getitem__(0, set_ans="yes")

print(prompt)
print(answer)

predict_next_token(lm, prompt, k = 5)

[{'mug': 'milk', 'cup': 'juice'}, {'mug': 'milk', 'cup': 'juice'}]
Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer:" tag.

Story: Alice is working in a busy restaurant. A customer asks Alice for milk. Alice grabs an opaque mug and fills it with milk. Then Alice grabs another opaque cup and fills it with juice.
Question: Does Alice believe cup contains juice??
Answer:
yes


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


[PredictedToken(token=' YES', prob=0.6680120229721069, logit=20.078125, token_id=14410),
 PredictedToken(token=' Yes', prob=0.13785207271575928, logit=18.5, token_id=7566),
 PredictedToken(token=' yes', prob=0.0947442576289177, logit=18.125, token_id=10035),
 PredictedToken(token=' NO', prob=0.04270457848906517, logit=17.328125, token_id=5782),
 PredictedToken(token=' No', prob=0.02081255242228508, logit=16.609375, token_id=2360)]

In [7]:
sample = SampleV3(
    template = STORY_TEMPLATES["templates"][0],
    characters=["Alice", "Bob"],
    containers=["mug", "cup"],
    states=["milk", "juice"],
    event_idx = 0, # causal event happened
)

print(sample.character_belief)

d = DatasetV3(samples=[sample])
prompt, answer = d.__getitem__(0, set_character=0, set_container=0, set_state=0)

print(prompt)
print(answer)

predict_next_token(lm, prompt, k = 5)

[{'mug': 'milk', 'cup': 'juice'}, {'mug': 'juice', 'cup': 'juice'}]
Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer:" tag.

Story: Alice is working in a busy restaurant. A customer asks Alice for milk. Alice grabs an opaque mug and fills it with milk. Then Alice grabs another opaque cup and fills it with juice. A co-worker named Bob didn't hear the customer's request and swaps the milk in the mug with juice while Alice was attending to another task. Alice didn't notice Bob swapping the contents of the mug.
Question: Does Alice believe mug contains milk??
Answer:
yes


[PredictedToken(token=' YES', prob=0.5779160857200623, logit=20.8125, token_id=14410),
 PredictedToken(token=' yes', prob=0.2729882299900055, logit=20.0625, token_id=10035),
 PredictedToken(token=' Yes', prob=0.11925971508026123, logit=19.234375, token_id=7566),
 PredictedToken(token='Yes', prob=0.006728173699229956, logit=16.359375, token_id=9642),
 PredictedToken(token='yes', prob=0.006521169096231461, logit=16.328125, token_id=9891)]

In [8]:
# synthetic entities => filter to single token entities

from src.functional import prepare_input

TYPE = "states"
ROOT = os.path.join(
    env_utils.DEFAULT_DATA_DIR, "synthetic_entities", TYPE
)

for file in os.listdir(ROOT):
    file_path = os.path.join(ROOT, file)
    # print(file)
    with open(file_path, "r") as f:
        names = json.load(f)
    single_token_entities = []
    for name in names:
        tok = prepare_input(prompts=f" {name}", tokenizer=lm)
        if tok.input_ids.shape[1] == 2 and tok.input_ids[0][0] == lm.tokenizer.bos_token_id:
            single_token_entities.append(name)
    
    print(f"{file}: filtered {len(single_token_entities)}/{len(names)}")
    assert len(single_token_entities) >= 2

    with open(file_path, "w") as f:
        json.dump(single_token_entities, f, indent=4)

cake_ingridients.json: filtered 15/15
gemstones.json: filtered 6/6
evidence.json: filtered 15/15
magic_objs.json: filtered 15/15
seeds.json: filtered 7/7
chemicals.json: filtered 8/8
book.json: filtered 9/9
drinks.json: filtered 30/30
artifact.json: filtered 8/8
plants.json: filtered 6/6


In [20]:
import itertools, random

all_states = {}
all_containers= {}
all_characters = json.load(open(os.path.join(env_utils.DEFAULT_DATA_DIR, "synthetic_entities", "characters.json"), "r"))



for TYPE, DCT in {"states": all_states, "containers": all_containers}.items():
    ROOT = os.path.join(
        env_utils.DEFAULT_DATA_DIR, "synthetic_entities", TYPE
    )
    for file in os.listdir(ROOT):
        file_path = os.path.join(ROOT, file)
        with open(file_path, "r") as f:
            names = json.load(f)
        DCT[file.split(".")[0]] = names


for template in STORY_TEMPLATES["templates"]:
    characters = random.sample(all_characters, 2)
    containers = random.sample(all_containers[template["container_type"]], 2)
    states = random.sample(all_states[template["state_type"]], 2)
    for event_idx in [None, 0, 1]:

        event_noticed = False if event_idx is None else random.choice([True, False])

        sample = SampleV3(
            template = template,
            characters=characters,
            containers=containers,
            states=states,
            event_idx = event_idx, # causal event happened
            event_noticed = event_noticed
        )
        prompt, answer = DatasetV3(samples=[sample]).__getitem__(0, set_character=0, set_container=0, set_state=0)

        print("-"*50)
        print(prompt)
        print(answer)
        pred = predict_next_token(lm, prompt, k = 5)
        print(pred)

    print("#"*50)
    print("\n\n")

--------------------------------------------------
Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer:" tag.

Story: Tim is working in a busy restaurant. A customer asks Tim for espresso. Tim grabs an opaque dispenser and fills it with espresso. Then Tim grabs another opaque pitcher and fills it with juice.
Question: Does Tim believe dispenser contains espresso??
Answer:
yes
[PredictedToken(token=' YES', prob=0.5348826050758362, logit=22.46875, token_id=14410), PredictedToken(token=' yes', prob=0.31939297914505005, logit=21.953125, token_id=10035), PredictedToken(token=' Yes', prob=0.12904644012451172, logit=21.046875, token_id=7566), PredictedToken(token='Yes', prob=0.0064248437993228436, logit=18.046875, token_id=9642), PredictedToken(token='yes', prob=0.0057591930963099, logit=17.9375,

In [57]:
LIMIT = 10000

configs = []

while(len(configs) < LIMIT):
    template_idx = random.choice(range(len(STORY_TEMPLATES["templates"])))
    template = STORY_TEMPLATES["templates"][template_idx]
    characters = random.sample(all_characters, 2)
    containers = random.sample(all_containers[template["container_type"]], 2)
    states = random.sample(all_states[template["state_type"]], 2)

    event_idx = random.choices([None, 0, 1], weights=[0.4, 0.5, 0.1], k=1)[0]
    event_noticed = False if event_idx is None else random.choices([False, True], weights=[0.8, 0.2], k = 1)[0]

    configs.append((
        template_idx, 
        characters[0], characters[1],
        containers[0], containers[1],
        states[0], states[1],
        event_idx, event_noticed
    ))

    if len(configs) % 1000 == 0:
        pre_len = len(configs)
        configs = list(set(configs))
        print(f"Filtered {len(configs)}/{pre_len}")


Filtered 1000/1000
Filtered 2000/2000
Filtered 3000/3000
Filtered 4000/4000
Filtered 5000/5000
Filtered 6000/6000
Filtered 7000/7000
Filtered 8000/8000
Filtered 9000/9000
Filtered 10000/10000


In [58]:
from src.dataset import DatasetV3

samples: list[SampleV3] = []
for config in configs:
    samples.append(
        SampleV3(
            template = STORY_TEMPLATES["templates"][config[0]],
            characters=[config[1], config[2]],
            containers=[config[3], config[4]],
            states=[config[5], config[6]],
            event_idx = config[7], # causal event happened
            event_noticed = config[8]
        )
    )

dataset = DatasetV3(samples=samples)
len(dataset)

10000

In [59]:
with open(
    os.path.join(env_utils.DEFAULT_DATA_DIR, "dataset_v4.json"), "w") as f:
    json.dump(dataset.to_dict(), f
)

In [60]:
with open(
    os.path.join(env_utils.DEFAULT_DATA_DIR, "dataset_v4.json"), "r"
) as f:
    dataset_dict = json.load(f)

In [61]:
sample_dict = dataset_dict["samples"][5]

sample = SampleV3.from_dict(sample_dict)

In [62]:
loaded_dataset = DatasetV3.from_dict(dataset_dict)

In [10]:
# import itertools
# import random

# root = os.path.join(
#     env_utils.DEFAULT_DATA_DIR, "synthetic_entities"
# )
# actors = json.load(open(os.path.join(root, "actor.json")))
# objects = json.load(open(os.path.join(root, "object.json")))
# containers = json.load(open(os.path.join(root, "container.json")))

# actors = list(set(actors))
# objects = list(set(objects))
# containers = list(set(containers))

# len(actors), len(objects), len(containers)

In [11]:
# actorsC2 = list(itertools.combinations(actors, 2))
# objectsC2 = list(itertools.combinations(objects, 2))
# containersC2 = list(itertools.combinations(containers, 2))

In [12]:
# limit = 10000

# configs = []
# while len(configs) < limit:
#     protagonist, perpetrator = random.choice(actorsC2)
#     object1, object2 = random.choice(objectsC2)
#     container1, container2 = random.choice(containersC2)

#     # changing th econtents of the second container has not related to the customer's order.
#     event_idx = random.choices([0,1], weights=[0.6, 0.4], k = 1)[0]

#     # protagonists belief != true state only if event_noticed == False
#     event_noticed = random.choices([False, True], weights=[0.7, 0.3], k = 1)[0]

#     configs.append((
#         protagonist, perpetrator,
#         object1, object2,
#         container1, container2,
#         event_idx, event_noticed
#     ))

#     if len(configs) % 1000 == 0:
#         pre_len = len(configs)
#         configs = list(set(configs))
#         print(f"filtered to {len(configs)} samples from {pre_len}")

In [13]:
# from src.dataset import SampleV3, DatasetV3
# samples: list[SampleV3] = []
# for config in configs:
#     sample = SampleV3(
#         protagonist=config[0],
#         perpetrator=config[1],
#         objects=[config[2], config[3]],
#         containers=[config[4], config[5]],
#         event_idx=config[6],
#         event_noticed=config[7]
#     )
#     samples.append(sample)

# dataset = DatasetV3(samples=samples)
# len(dataset)

In [14]:
# # question, answer = dataset[2]
# question, answer = dataset.__getitem__(2, set_ans="yes")
# print(question)
# print(answer)

In [15]:
# from src.utils import env_utils

# with open(
#     os.path.join(env_utils.DEFAULT_DATA_DIR, "dataset_v3.json"), "w") as f:
#     json.dump(dataset.to_dict(), f
# )

In [16]:
# with open (os.path.join(env_utils.DEFAULT_DATA_DIR, "dataset_v3.json"), "r") as f:
#     dataset_dict = json.load(f)

# loaded_dataset = DatasetV3.from_dict(dataset_dict)
# len(loaded_dataset)

In [17]:
# from scripts.collect_binding_id_states import ExperimentResults

# cache_path = os.path.join(
#     env_utils.DEFAULT_RESULTS_DIR, 
#     "binding_id_states",
#     "Meta-Llama-3-70B-Instruct", 
#     "results.json"
# )
# with open(cache_path, "r") as f:
#     results = json.load(f)

In [18]:
# res = ExperimentResults.from_dict(results)

In [19]:
# res.cached_states[5].context_informed_actor['model.layers.79_<>_155'].shape